In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import dlib
import cv2
import os
import numpy as np
from PIL import Image, ImageChops, ImageEnhance
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import statistics
from statistics import mode

In [ ]:
tf.__version__

'2.9.2'

In [ ]:
model = load_model('./model/model-deepfake-detection.h5')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 2, 2, 1536)       54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 2)                 3074      
                                                                 
Total params: 54,339,810
Trainable params: 54,279,266
Non-trainable params: 60,544
_________________________________________________________________


In [ ]:
from google.colab.patches import cv2_imshow
from tensorflow import keras

from PIL import Image
from PIL import ImageFilter
from PIL import ImageEnhance


In [ ]:
def sp_hehe(model , img):
  pre_lst = []
  for i in img:
    data = img_to_array(cv2.resize(i, (128, 128))).flatten() / 255.0
    data = data.reshape(-1, 128, 128, 3)
    pre = model.predict(data)
    if ((pre > 0.5).astype("int32")[0][0] == 0 ):
      pre_lst.append(1)
    else :
      pre_lst.append(0)

  return pre_lst

In [ ]:
def prediction (filepath):
  input_shape = (128, 128, 3)
  pr_data = []
  detector = dlib.get_frontal_face_detector()
  cap = cv2.VideoCapture(filepath)
  frameRate = cap.get(5)
  print(frameRate)
  real_frame = 0
  fake_frame = 0
  sp_1 = 3
  sp_2 = 4
  while cap.isOpened():
      frameId = cap.get(1)
      ret, frame = cap.read()
      if ret != True:
          break
      if round(frameId *  2) % round(frameRate) == 0:
        face_rects, scores, idx = detector.run(frame, 0)
        for i, d in enumerate(face_rects):
              x1 = d.left()
              y1 = d.top()
              x2 = d.right()
              y2 = d.bottom()
              crop_img = frame[y1:y2, x1:x2]
              if crop_img.shape[0] != 0 and crop_img.shape[1] != 0:
                flag = 1
                grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
                laplacian_var = cv2.Laplacian(grey, cv2.CV_64F).var()
                crop_img_2 = crop_img.copy()
                crop_img_3 = crop_img.copy()
                crop_img_4 = crop_img.copy()
                crop_img_5 = crop_img.copy()
                laplacian_var_3 =laplacian_var
                laplacian_var_4 = laplacian_var
                laplacian_var_5 = laplacian_var
                if laplacian_var < 25:
                  flag =5
                elif laplacian_var < 50:
                  flag = 4
                elif laplacian_var <75:
                  flag = 3
                elif laplacian_var <100:
                  flag =2
                else:
                  flag =1
                while  laplacian_var_3 < 25:
                  kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]], np.float32)
                  crop_img_3 = cv2.filter2D(crop_img_3, -1, kernel)
                  grey = cv2.cvtColor(crop_img_3, cv2.COLOR_BGR2GRAY)
                  laplacian_var_3 = cv2.Laplacian(grey, cv2.CV_64F).var()
                while laplacian_var_4 < 50:
                  crop_img_4= cv2.detailEnhance(crop_img_4, sigma_s=10, sigma_r=0.15)
                  crop_img_4= Image.fromarray(crop_img_4)
                  crop_img_4 = crop_img_4.filter(ImageFilter.SHARPEN)
                  crop_img_4 = np.array(crop_img_4)
                  grey = cv2.cvtColor(crop_img_4, cv2.COLOR_BGR2GRAY)
                  laplacian_var_4 = cv2.Laplacian(grey, cv2.CV_64F).var()
                while laplacian_var_5 <75:
                  crop_img_5= cv2.detailEnhance(crop_img_5, sigma_s=10, sigma_r=0.15)
                  grey = cv2.cvtColor(crop_img_5, cv2.COLOR_BGR2GRAY)
                  laplacian_var_5 = cv2.Laplacian(grey, cv2.CV_64F).var()
                while laplacian_var < 100:
                    crop_img= Image.fromarray(crop_img)
                    crop_img = crop_img.filter(ImageFilter.SHARPEN)
                    crop_img = np.array(crop_img)
                    grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
                    laplacian_var = cv2.Laplacian(grey, cv2.CV_64F).var()
                if flag ==5 :
                  img = [crop_img,crop_img_2,crop_img_3,crop_img_4,crop_img_5]
                  pre = sp_hehe(model,img)
                if flag == 4:
                  img = [crop_img,crop_img_2,crop_img_4,crop_img_5]
                  pre = sp_hehe(model,img)
                if flag == 3:
                  img = [crop_img,crop_img_2,crop_img_5]
                  pre = sp_hehe(model,img)
                if flag == 2:
                  img = [crop_img,crop_img_2]
                  pre = sp_hehe(model,img)
                if flag == 1:
                  img = [crop_img_2]
                  pre = sp_hehe(model,img)
                outt=''
                if flag %2 ==0 :
                  if pre.count(1) != pre.count(0):
                    if pre.count(1) > pre.count(0):
                      real_frame+=1
                    else:
                      fake_frame+=1
                else:
                  if flag !=1:
                    if sum(pre) >= int(flag/2) +1:
                      real_frame+=1
                    else:
                      fake_frame+=1
                  else:
                    if pre[0] == 1:
                      real_frame+=1
                    else:
                      fake_frame+=1


  print(real_frame)
  print(fake_frame)
  if (real_frame == 0 and fake_frame == 0):
    return 'Real'
  elif  fake_frame-real_frame == 1 :
    return 'Fake'
  elif ( real_frame/fake_frame > 1.065) or( real_frame/fake_frame >= 0.98 and real_frame/fake_frame < 1):
    return 'Real'
  else:
    return'Fake'

In [ ]:
import os
path = "./video up"
dir_list = os.listdir(path)

In [ ]:
lst_real_fake= []
for i in dir_list:
  lst_real_fake.append(prediction(path+"/"+i))

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 237ms/step
78
166
23.976023976023978
1/1 [==============================] - 0s 150ms/step
46
65
23.976023976023978
1/1 [==============================] - 0s 149ms/step
180
211
23.976023976023978
1/1 [==============================] - 0s 141ms/step
77
159
30.0
1/1 [==============================] - 0s 142ms/step
52
71
24.0
1/1 [==============================] - 0s 154ms/step
46
69
23.976023976023978
1/1 [==============================] - 0s 148ms/step
55
66
30.0
1/1 [==============================] - 0s 146ms/step
107
65
60.0
1/1 [==============================] - 0s 137ms/step
38
34
30.0
1/1 [==============================] - 0s 138ms/step
184
234
29.97002997002997
1/1 [==============================] - 0s 145ms/step
4
2
30.0
1/1 [==============================] - 0s 144ms/step
107
109
29.97002997002997
1/1 [==============================] - 0s 141ms/step
61
56


In [ ]:
for i in range (0,len(lst_real_fake)):
  print(dir_list[i].split('.')[0]+ ": "+ lst_real_fake[i])

Artists create Zuckerberg 'deepfake' video: Fake
Donald Trump is Fat Bastard! (DeepFake): Fake
Neo Takes The Blue Pill [DeepFake] (1): Real
Chris Bumstead vs Breon Ansley - -They Are Fighting For 2nd Place- - 2021 Mr: Real
Jim Carrey Walks Into A Bar [DeepFake]: Fake
Jim Carrey as Joker [deepfake]: Fake
Better Call Trump- Money Laundering 101 [DeepFake]: Fake
Hôm Nay Em Cưới Rồi - Khải Đăng - Official MV- Đàm Vĩnh Hưng, Vũ Hà, Ribi Sachi, Pom, Jay Quân: Fake
Chúng Ta Không Thuộc Về Nhau - Official Music Video - Sơn Tùng M-TP: Real
So_sanh_Tokbokki_pho_mai_an_lien_beo_ngay-_hang_O_hay_hang_Y_eu_noi_tieng-_-_TRUONG_TRUE_shorts: Real
Luis Fonsi - Despacito ft: Real
Ed Sheeran - Shape of You (Official Music Video): Real
Wiz Khalifa - See You Again ft: Real
Maroon 5 - Sugar (Official Music Video): Real
OneRepublic - Counting Stars (Official Music Video): Fake
Bill Hader impersonates Arnold Schwarzenegger [DeepFake]: Fake
The Office - Identity Theft [DeepFake]: Fake
Dee